In [ ]:
import pandas as pd
import numpy as np
import json

def simulation(raw_df, coef_data, input_DOD, act_DOD, multi_data, reg_price, dod, unitcost):
    # Round and log-transform input data
    input_DOD = round(input_DOD, 2)
    input_DOD1 = input_DOD.copy()
    input_DOD[reg_price] = np.log(input_DOD[reg_price])
    input_DOD.set_index(['Mother_SKU'], inplace=True)

    act_DOD = round(act_DOD, 2)
    act_DOD1 = act_DOD.copy()
    act_DOD[reg_price] = np.log(act_DOD[reg_price])
    act_DOD.set_index(['Mother_SKU'], inplace=True)

    # Prepare raw data and coefficient data
    Z = raw_df[['Mother_SKU', 'Date', 'Quarter']]
    unit_cost = raw_df.groupby(['Mother_SKU'])[unitcost].mean().reset_index()
    raw_df.set_index(['Mother_SKU', 'Date', 'Quarter'], inplace=True)
    
    coef_data.rename(columns={'Regular Price Variable': reg_price, 'DOD Variable': dod}, inplace=True)
    merged_coef = Z.merge(coef_data, on=['Mother_SKU'], how='left')
    merged_coef.set_index(['Mother_SKU', 'Date', 'Quarter'], inplace=True)
    merged_coef = merged_coef.reindex(index=raw_df.index)

    # Calculate historical predictions
    Historical_pre = pd.DataFrame()
    common_columns = merged_coef.columns.intersection(raw_df.columns)
    for column_name in common_columns:
        if pd.api.types.is_numeric_dtype(merged_coef[column_name]):
            Historical_pre[column_name] = merged_coef[column_name] * raw_df[column_name]

    # Simulate future values
    dod_columns = Historical_pre.columns[Historical_pre.columns.str.endswith("_DOD")]
    dod_df = pd.DataFrame(1, index=multi_data.index, columns=dod_columns)
    multi_data = pd.concat([multi_data, dod_df], axis=1)

    Pre = pd.DataFrame()
    Pre['XB'] = Historical_pre.sum(axis=1)
    Pre['base'] = np.log(raw_df['Predicted']) - Pre['XB']
    
    multi_dod_reg = input_DOD / act_DOD
    df = multi_dod_reg.reset_index()
    melted_df = df.melt(id_vars=['Mother_SKU', reg_price], var_name='Quarter', value_name=dod)
    melted_df = melted_df.sort_values(by=['Mother_SKU', 'Quarter'])
    melted_df1 = melted_df[['Mother_SKU', 'Quarter', dod, reg_price]].reset_index(drop=True)

    merged_multi1 = multi_data.merge(melted_df1, on=['Mother_SKU', 'Quarter'], how='left')
    merged_multi = Z.merge(merged_multi1, on=['Mother_SKU', 'Quarter'], how='left')
    merged_multi.set_index(['Mother_SKU', 'Date', 'Quarter'], inplace=True)

    Simulated_con = Historical_pre * merged_multi
    Simulated = pd.DataFrame()
    Simulated_Vol = pd.DataFrame()
    Simulated['Base'] = Pre['base']
    Simulated['XB'] = Simulated_con.sum(axis=1)
    Simulated_Vol['Volume'] = np.exp(Simulated['Base'] + Simulated['XB'])

    Simulated_Vol = Simulated_Vol.reset_index()
    Simulated_Vol = Simulated_Vol[['Mother_SKU', 'Quarter', 'Volume']]
    Simulated_Vol = Simulated_Vol.groupby(['Mother_SKU', 'Quarter'])['Volume'].sum().reset_index()

    # Calculate financial metrics
    unit_cost.set_index(['Mother_SKU'], inplace=True)
    unit_cost = unit_cost.reindex(index=act_DOD.index)

    pre_vol = raw_df['Predicted'].reset_index()
    pre_vol = pre_vol[['Mother_SKU', 'Quarter', 'Predicted']]
    pre_vol = pre_vol.groupby(['Mother_SKU', 'Quarter'])['Predicted'].sum().reset_index()

    melted_df1 = input_DOD1.melt(id_vars=["Mother_SKU", reg_price], var_name="Quarter", value_name="Fut_DOD")
    melted_df2 = act_DOD1.melt(id_vars=['Mother_SKU', reg_price], var_name='Quarter', value_name='His_DOD')

    melted_df1 = melted_df1.sort_values(by=['Mother_SKU', 'Quarter']).reset_index(drop=True)
    melted_df2 = melted_df2.sort_values(by=['Mother_SKU', 'Quarter']).reset_index(drop=True)

    calculation = melted_df1.merge(melted_df2, on=['Mother_SKU', 'Quarter'], how='left')
    calculation = calculation.merge(unit_cost, on=['Mother_SKU'], how='left')
    calculation = calculation.merge(pre_vol, on=['Mother_SKU', 'Quarter'], how='left')
    calculation = calculation.merge(Simulated_Vol, on=['Mother_SKU', 'Quarter'], how='left')

    calculation['Avg_selling_price'] = calculation[f'{reg_price}_x'] * (1 - calculation['His_DOD'])
    calculation['Fut_selling_price'] = calculation[f'{reg_price}_y'] * (1 - calculation['Fut_DOD'])
    calculation['Base_Revenue'] = calculation['Predicted'] * calculation['Avg_selling_price']
    calculation['Future_Revenue'] = calculation['Volume'] * calculation['Fut_selling_price']
    calculation['Base_Margin'] = calculation['Predicted'] * (calculation['Avg_selling_price'] - calculation[unitcost])
    calculation['Future_Margin'] = calculation['Volume'] * (calculation['Fut_selling_price'] - calculation[unitcost])
    calculation['Base Investment'] = calculation['Predicted'] * (calculation[f'{reg_price}_x'] - calculation['Avg_selling_price'])
    calculation['Future Investment'] = calculation['Volume'] * (calculation[f'{reg_price}_y'] - calculation['Fut_selling_price'])

    Final_calculation = pd.DataFrame()
    Final_calculation[['Mother_SKU', 'PREDICTED_Volume', 'Future_Volume', 'Base_Revenue', 'Future_Revenue', 'Base_Margin', 'Future_Margin', 'Base Investment', 'Future Investment']] = calculation[['Mother_SKU', 'Predicted', 'Volume', 'Base_Revenue', 'Future_Revenue', 'Base_Margin', 'Future_Margin', 'Base Investment', 'Future Investment']]

    sim_result = Final_calculation.groupby(['Mother_SKU']).sum().reset_index()
    sim_result = round(sim_result, 2)

    # Original overall metrics calculation
    overall_metrics = [
        {
            "KPIs": "Revenue",
            "Baseline": sim_result['Base_Revenue'].sum(),
            "Future_State_Scenario": sim_result['Future_Revenue'].sum(),
            "Change": round(((sim_result['Future_Revenue'].sum() - sim_result['Base_Revenue'].sum()) / sim_result['Base_Revenue'].sum()) * 100)
        },
        {
            "KPIs": "Gross Margin",
            "Baseline": sim_result['Base_Margin'].sum(),
            "Future_State_Scenario": sim_result['Future_Margin'].sum(),
            "Change": round(((sim_result['Future_Margin'].sum() - sim_result['Base_Margin'].sum()) / sim_result['Base_Margin'].sum()) * 100)
        },
        {
            "KPIs": "Volume",
            "Baseline": sim_result['PREDICTED_Volume'].sum(),
            "Future_State_Scenario": sim_result['Future_Volume'].sum(),
            "Change": round(((sim_result['Future_Volume'].sum() - sim_result['PREDICTED_Volume'].sum()) / sim_result['PREDICTED_Volume'].sum()) * 100)
        },
        {
            "KPIs": "Investment",
            "Baseline": sim_result['Base Investment'].sum(),
            "Future_State_Scenario": sim_result['Future Investment'].sum(),
            "Change": round(((sim_result['Future Investment'].sum() - sim_result['Base Investment'].sum()) / sim_result['Base Investment'].sum()) * 100)
        }
    ]

    # Collect individual quarter results
    individual_quarters = calculation[['Mother_SKU', 'Quarter', 'Base_Revenue', 'Future_Revenue', 'Base_Margin', 'Future_Margin', 'Predicted', 'Volume', 'Base Investment', 'Future Investment']].copy()
    individual_quarters.columns = ['Mother_SKU', 'Quarter', 'Base_Revenue', 'Future_Revenue', 'Base_Margin', 'Future_Margin', 'PREDICTED_Volume', 'Future_Volume', 'Base_Investment', 'Future_Investment']
    individual_quarters = round(individual_quarters, 2)

    # Prepare data for Graph1 and Graph2
    graph1_data = {
        "Base_Revenue": [],
        "Future_Revenue": [],
        "Base_Volume": [],
        "Future_Volume": []
    }
    
    graph2_data = {
        "Base_Margin": [],
        "Future_Margin": [],
        "Base_Investment": [],
        "Future_Investment": []
    }

    quarters = sorted(individual_quarters['Quarter'].unique())

    for quarter in quarters:
        base_revenue = individual_quarters[individual_quarters['Quarter'] == quarter]['Base_Revenue'].sum()
        future_revenue = individual_quarters[individual_quarters['Quarter'] == quarter]['Future_Revenue'].sum()
        base_volume = individual_quarters[individual_quarters['Quarter'] == quarter]['PREDICTED_Volume'].sum()
        future_volume = individual_quarters[individual_quarters['Quarter'] == quarter]['Future_Volume'].sum()
        
        base_margin = individual_quarters[individual_quarters['Quarter'] == quarter]['Base_Margin'].sum()
        future_margin = individual_quarters[individual_quarters['Quarter'] == quarter]['Future_Margin'].sum()
        base_investment = individual_quarters[individual_quarters['Quarter'] == quarter]['Base_Investment'].sum()
        future_investment = individual_quarters[individual_quarters['Quarter'] == quarter]['Future_Investment'].sum()

        graph1_data["Base_Revenue"].append(base_revenue)
        graph1_data["Future_Revenue"].append(future_revenue)
        graph1_data["Base_Volume"].append(base_volume)
        graph1_data["Future_Volume"].append(future_volume)

        graph2_data["Base_Margin"].append(base_margin)
        graph2_data["Future_Margin"].append(future_margin)
        graph2_data["Base_Investment"].append(base_investment)
        graph2_data["Future_Investment"].append(future_investment)

    result = {
        "Overall_Metrics": overall_metrics,
        "Graph1": {
            "Base_Revenue": graph1_data["Base_Revenue"],
            "Future_Revenue": graph1_data["Future_Revenue"],
            "Base_Volume": graph1_data["Base_Volume"],
            "Future_Volume": graph1_data["Future_Volume"]
        },
        "Graph2": {
            "Base_Margin": graph2_data["Base_Margin"],
            "Future_Margin": graph2_data["Future_Margin"],
            "Base_Investment": graph2_data["Base_Investment"],
            "Future_Investment": graph2_data["Future_Investment"]
        }
    }

    json_result = json.dumps(result)

    return json_result
